In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import datetime as dt

## Code to create dataset with outages and weather information

Paths to folders

In [14]:
output_folder = 'temp_2'
unclean_data_folder = 'historical_data_raw'
weather_data = 'weather_data'

Path(output_folder).mkdir(exist_ok=True)
Path(unclean_data_folder).mkdir(exist_ok=True)
Path(weather_data).mkdir(exist_ok=True)

Creates clean csv files with relevant information for each annual summary

In [15]:
def split_area(df):
    df_states_only = df[~df['Area'].str.contains('County|Parish', na=False)]
    df_county_only = df[df['Area'].str.contains('County|Parish', na=False)]

    states_split = df_states_only['Area'].str.split(r': |, ',regex=True)
    counties_split = df_county_only['Area'].str.split(r'; |: ', regex=True) # Special case for mulitiple states in a single row

    # Appends county count to end of state string (ex Alabama:2)
    def count_counties(x):
        num_counties = len(x[1].split(', '))
        x[0] = x[0] + f':{num_counties}'
        if ', ' in x[0]:
            x[0] = x[0].split(', ')[-1]
        return [x[0]]

    counties_split = counties_split.apply(count_counties)
    df['Area'] = pd.concat([states_split, counties_split])

    # Expands each element of the list into its own row
    df = df.explode('Area', ignore_index=True)

    # Cleans string
    def custom_clean(x):
        if x[-1] in [';', ':']:
            x = x[:-1]
        if x[-1] == ']':
            x = x[:x.index('[')]
        return x
    df['Area'] = df['Area'].apply(custom_clean)
    df = df.rename(columns={"Area": "State"})
    df['State'] = df['State'].apply(lambda x: x + ':-1' if ':' not in x else x)
    df['Num Counties Affected'] = df['State'].apply(lambda x: x.split(':')[1])
    df['State'] = df['State'].apply(lambda x: x.split(':')[0])

    df['Num Counties Affected'] = df['Num Counties Affected'].apply(pd.to_numeric)
    df['Num Counties Affected'] = df['Num Counties Affected'].apply(lambda x: np.nan if x == -1 else x)

    return df

def clean_data(df):
    '''Cleans annual summary files to save relevant data'''

    # Isolate outages caused by weather and remove unusable/unecessary data
    df = df[df['Event Type'].str.contains('Weather', na=False)]
    df = df[~df['Date of Restoration'].astype(str).str.contains('Unknown', na=False)]
    df.drop(columns=['Month', 'Alert Criteria', 'Event Type', 'Demand Loss (MW)', 'NERC Region'], inplace=True, errors='ignore')
    df.dropna(axis='columns', how='all', inplace=True)

    # Convert dates and times to consistent format
    df['Date Event Began'] = pd.to_datetime(df['Date Event Began'])
    df['Date Event Began'] = df['Date Event Began'].dt.date
    df['Date of Restoration'] = pd.to_datetime(df['Date of Restoration'])
    df['Date of Restoration'] = df['Date of Restoration'].dt.date
    df['Time Event Began'] = pd.to_datetime(df['Time Event Began'].astype(str))
    df['Time Event Began'] = df['Time Event Began'].dt.time
    df['Time of Restoration'] = pd.to_datetime(df['Time of Restoration'].astype(str))
    df['Time of Restoration'] = df['Time of Restoration'].dt.time

    # Rename cols for clarity
    df = df.rename(columns={"Date Event Began": "Start Date", "Time Event Began": "Start Time", 
        "Date of Restoration": "End Date", "Time of Restoration": "End Time", "Area Affected": "Area"})

    # Create dest filepath if it doesn't exist and save to csv
    df = split_area(df)
    df['Number of Customers Affected'] = pd.to_numeric(df['Number of Customers Affected'], errors='coerce')

    return df

files = Path(f'{unclean_data_folder}/').glob('*_Annual_Summary.xls')
for file in sorted(files):
    name = file.name[:-4]
    df = pd.read_excel(f'{unclean_data_folder}/{name}.xls', header=1)
    df = clean_data(df)
    df.to_csv(f'{output_folder}/{name}.csv', index=False)

Code to remove time cols from annual summary files

In [16]:
files = Path(f'{output_folder}/').glob('*_Annual_Summary.csv')
for file in sorted(files):
    name = file.name
    df = pd.read_csv(f'{output_folder}/{name}')
    df.drop(columns=['Start Time', 'End Time'], inplace=True, errors='ignore')
    df['Date'] = [pd.date_range(s, e, freq='d') for s, e in
              zip(pd.to_datetime(df['Start Date']),
                  pd.to_datetime(df['End Date']))]
    df = df.explode('Date', ignore_index=True).drop(['Start Date', 'End Date'], axis=1)
    df = df[['Date', 'State', 'Number of Customers Affected', 'Num Counties Affected']]
    df.to_csv(Path(f'{output_folder}/{name}'), index=False)

Code to create merged dataset

In [17]:
data_list = Path(f'{output_folder}/').glob('*_Annual_Summary.csv')
lst = sorted([str(path) for path in data_list])
df = pd.concat(map(pd.read_csv, lst), ignore_index=True)
df.to_csv(Path(f'{output_folder}/' + 'merged_data.csv'), index=False)

Code to create seperate csvs for each state

Block 1 - CSVs for states in dataframe
Block 2 - CSVs for states not in dataframe

In [18]:
df = pd.read_csv(f'{output_folder}/merged_data.csv')
for state in df.State.unique():
    state_format = state.replace(' ', '_')
    state_df = df.loc[df['State'] == state]
    file = f'{output_folder}/states/{state_format}.csv'
    path = Path(file)
    path.parents[0].mkdir(parents=True, exist_ok=True)
    state_df.to_csv(file, index=False)

In [19]:
unkown_states = ['Alaska', 'Hawaii', 'Montana', 'New_Mexico', 'Utah', 'Wyoming']
for state_name in unkown_states:
    state_df = pd.DataFrame(columns=['Date','State','Number of Customers Affected','Num Counties Affected'])
    file = f'{output_folder}/states/{state_name}.csv'
    state_df.to_csv(file, index=False)

Code to expand dates, label outage days per state, and add season feature

In [20]:
start_date = dt.datetime(year=2017, month=1, day=1).date()
end_date = dt.datetime(year=2021, month=6, day=21).date()
seasons = ['Winter', 'Winter', 'Spring', 'Spring', 'Spring', 'Summer', 'Summer', 'Summer', 'Fall', 'Fall', 'Fall', 'Winter']
month_to_season = dict(zip(range(1,13), seasons))


files = Path(f'{output_folder}/states/').glob('*.csv')
for file in files:
    name = f'{output_folder}/states/{file.name}'
    state = file.name[0:-4]
    state = state.replace('_', ' ')
    df = pd.read_csv(f'{name}')
    df['Outage'] = 1
    df['Date'] = pd.to_datetime(df['Date'])

    df = df.set_index('Date')

    if start_date not in df.index:
        row = {'State': state, 'Outage': 0}
        temp = pd.DataFrame(data=row, index=[pd.to_datetime(start_date)])
        df = pd.concat([df, temp])
    if end_date not in df.index:
        row = {'State': state, 'Outage': 0}
        temp = pd.DataFrame(data=row, index=[pd.to_datetime(end_date)])
        df = pd.concat([df, temp])

    df = df.sort_index() 

    df = df.resample('D').agg({'Outage': np.sum, 'Num Counties Affected': np.max, 'Number of Customers Affected': np.max})   
    df['State'] = state
    df = df[['State', 'Outage', 'Number of Customers Affected', 'Num Counties Affected']]
    df.index.name = 'Date'

    df['Outage'][df['Outage'] >= 1] = 1
    df['Number of Customers Affected'] = df.apply(lambda x: 0 if x['Outage'] == 0 else x['Number of Customers Affected'], axis=1)
    df['Num Counties Affected'] = df.apply(lambda x: 0 if x['Outage'] == 0 else x['Num Counties Affected'], axis=1)
    df['Season'] = df.index.month.map(month_to_season)

    df.to_csv(name)

Code to clean the unclean weather data csvs and append to state file

In [21]:
p = Path(f'{weather_data}/')
subdirectories = [x for x in p.iterdir() if x.is_dir()]
for folder in subdirectories:
    state = folder.name
    file = Path(f'{weather_data}/{state}/').glob('*_weather_new.csv')
    path = next(file)

    df = pd.read_csv(path)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df.drop(columns=['STATION'], inplace=True, errors='ignore')

    df = df.set_index('DATE')
    df = df.sort_index() 
    df = df.resample('D').mean()
    df.index.name = 'Date'
    df = df.rename(columns={'WT01': 'Fog', 'WT03': 'Thunder', 'WT05': 'Hail', 'WT07': 'Dust', 'WT10': 'Tornado', 'WT11': 
        'Wind', 'WT16': 'Rain', 'WT18': 'Snow'})
    df = df.apply(lambda x: x.fillna(0) if x.name in ['Fog', 'Thunder', 'Hail', 'Dust', 'Tornado', 'Wind', 'Rain', 'Snow'] else x)
    df.to_csv(f'{weather_data}/{state}/{path.name[:-4]}_clean.csv')

    state_path = f'{output_folder}/states/{state}.csv'
    state_df = pd.read_csv(f'{state_path}')
    state_df['Date'] = pd.to_datetime(state_df['Date'])
    fin = state_df.set_index('Date').join(df)
    fin.to_csv(Path(state_path), index=True)

Code to merge state weather files into a single file

In [122]:
data_list = Path(f'{output_folder}/states').glob('*.csv')
lst = sorted([str(path) for path in data_list])
df = pd.concat(map(pd.read_csv, lst), ignore_index=True)
df.to_csv(Path(f'{output_folder}/' + 'outage_n_weather_data.csv'), index=False)

## Links to manually download weather data

Code to get bounding boxes for each state

In [19]:
import requests
import pandas as pd

response = requests.get('https://gist.githubusercontent.com/a8dx/2340f9527af64f8ef8439366de981168/raw/81d876daea10eab5c2675811c39bcd18a79a9212/US_State_Bounding_Boxes.csv')
r = response.text

df = pd.DataFrame([x.split(',') for x in r.split('\n')])
df = df.rename(columns=df.iloc[0])
df.drop([0, 57, 3, 8, 11, 14, 43, 49], inplace=True) # Remove non US states
df.drop(['""'], axis=1, inplace=True)
df.columns = df.columns.str.replace('"','')
df['NAME'] = df.NAME.str.replace('"','')
df = df[["NAME", "ymax", "xmin", "ymin", "xmax"]]
df[['ymax', 'xmin', 'ymin', 'xmax']] = df[['ymax', 'xmin', 'ymin', 'xmax']].apply(pd.to_numeric)
df = df.reset_index(drop=True)


Code to get weather download url for each state

In [23]:
'''
For datatype information, go to 
https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/GHCND_documentation.pdf
'''

url = 'https://www.ncei.noaa.gov/access/search/data-search/daily-summaries'
params = {
    'dataTypes': ['AWND', 'PRCP', 'SNOW', 'SNWD', 'TMIN', 'TMAX', 'WT01', 'WT03', 'WT05', 'WT07', 'WT10', 'WT11', 'WT16', 'WT18'], 
    'startDate' : '2017-01-01T00:00:00', 
    'endDate': '2021-06-21T23:59:59', 
    'bbox': ''
}


with open('weather_urls.txt', 'w') as f:
    i = 1
    for index, row in df.iterrows():
        ymax = row['ymax']
        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        bbox = f"{ymax:.3f},{xmin:.3f},{ymin:.3f},{xmax:.3f}"
        params['bbox'] = bbox
        #print(bbox)
        r = requests.Request('GET', url, params=params)
        prep = r.prepare()
        f.write(row['NAME'] + '\n' + prep.url + '\n')
        if i % 10 == 0:
            f.write('\n')
        i += 1